In [ ]:
import numpy as np
import logging
#logging.getLogger("imperative_model").setLevel(logging.DEBUG)
#logging.basicConfig(level=logging.DEBUG)
from floweaver import weave, Dataset

from utils import update_sankey_data
from sankey_definitions import sdd, sdd_chemical_synthesis, sdd_primary, sdd_primary_detailed, palette
import load_model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
model_data = load_model.load_model()

In [ ]:
%run load_model_polymers.py
model, recipe_data = load_model.build_model(model_data)
recipe_data, other_results = define_polymer_model(
    model, 
    recipe_data, 
    processes_with_elec_req=[
        "DehydrogenationOfPropane",
        "SteamCrackingOfNaphthaMaxEthylene", 
        "SteamCrackingOfNaphthaMaxPropylene"
    ]
)

In [ ]:
flows_sym = model.to_flows(recipe_data, flow_ids=True)
func = model.lambdify(recipe_data)

# Work around dodgy floweaver behaviour -- flows index is reset when Dataset
# is created. This only works because not using dim_process etc.
dataset = Dataset(flows_sym)
dataset._flows = dataset._table = flows_sym.set_index("id")

In [ ]:
data = {}
data[Z_product] = [3] * len(product_names)
data[Z_EOL] = [1] * len(product_names)
data[RR] = [0.0] * len(polymer_objects)
data[a_PE_PP_mech_recyclable] = 1
data[C_ethyl_alcohol_from_biomass] = 0
data[C_ethylene_from_methyl_alcohol] = 20
data[C_xylenes_from_methyl_alcohol] = 0
data[C_xylenes_from_naphtha] = 5
data[C_green_hydrogen] = 0
data[C_blue_hydrogen] = 0
data[k_biomass_feedstock_fraction] = [1./6] * 6
data[Z_extra] = [1, 1, 1, 5]
data[k_olefins_from_paraffins_ethane_fraction] = 0.4

In [ ]:
# Do the floweaver calcs up front:
sankey_eqns = weave(sdd_primary_detailed, dataset, palette=palette)

w = update_sankey_data(sankey_eqns, func(data)).to_widget(width=1000, height=900, debugging=True)
w.children[0].scale = 2

# Uncomment this to show static Sankey using the data above
#w

Interactive Sankey diagram with sliders for parameters:

In [ ]:
from ipywidgets import interactive, IntSlider, HBox

def update_sankey(**params):
    # collapse vector params
    params["Z_extra"] = [params["ZeB"], params["ZeT"], params["ZeX"], params["ZeM"]]
    params["Z_product"] = [3] * len(product_names)
    params["Z_product"][1] = params["d1"]
    params["Z_EOL"] = params["Z_product"]
    params["RR"] = [params["rr"]] * len(polymer_objects)
    params["k_biomass_feedstock_fraction"] = [1./6] * 6

    new_w = update_sankey_data(sankey_eqns, func(params)).to_widget()
    if hasattr(w, "links"):
        w.links = new_w.links
    else:
        w.children[0].links = new_w.links  # debugging

ui = interactive(
    update_sankey,
    d1=(0., 10.), 
    rr=(0., 1.), 
    a_PE_PP_mech_recyclable=(0., 1.), 
    C_ethyl_alcohol_from_biomass=(0, 25), 
    C_ethylene_from_methyl_alcohol=(0, 25), 
    C_xylenes_from_methyl_alcohol=(0, 25), 
    C_xylenes_from_naphtha=(0, 25),
    C_blue_hydrogen=(0., 10.),
    C_green_hydrogen=(0., 10.),
    ZeB=(0., 10.),
    ZeT=(0., 10.),
    ZeX=(0., 10.),
    ZeM=(0., 10.),
    k_olefins_from_paraffins_ethane_fraction=(0., 1.)
)

In [ ]:
HBox([ui, w])